In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
from time import sleep
from datetime import datetime
from splinter import Browser
from selenium.webdriver.common.keys import Keys
import time
import json

In [2]:
# Path to chromedriver.exe
executable_path = {'executable_path':'C:/Users/zacha/AppData/Local/bin/chromedriver.exe'}

# Open the Splinter Browswer
browser = Browser('chrome', **executable_path)

In [3]:
# URL to find information on Coronavirus
url = 'https://www.dshs.state.tx.us/news/updates.shtm#coronavirus'
url2 = "http://www.google.com"

In [4]:
# Visiting the url 
browser.visit(url)

In [5]:
# Creating the beautiful soup to find the tables we need 
html = browser.html
soup = bs(html, 'html.parser')

In [6]:
# All tables on this webpage, this might be revised later
tables = soup.find_all('table')

In [7]:
# Selecting the table with all the Texas Counties
TX_counties = tables[2]
TX_counties = TX_counties.find_all('td')

In [8]:
# Creating lists of the counties and current number of cases per county

counties = []
cases = []

counter = 0 
for county in TX_counties:
    if counter % 2 == 0:
        counties.append(county.text)
        
    elif counter % 2 != 0:
        cases.append(county.text)
        
    counter = counter + 1
    

In [9]:
# Creating the first data frame. 
df = pd.DataFrame(list(zip(counties, cases)), columns =['Texas_County', 'COVID_19_Cases'])
df

,Texas_County,COVID_19_Cases
0,Bell,2
1,Bexar,12
2,Bowie,1
3,Brazoria,3
4,Brazos,2
5,Cameron,1
6,Collin,12
7,Crane,1
8,Dallas,22
9,Denton,6


In [10]:
# Going to collect lat long information on these Texas Counties. 
counties = df.Texas_County.tolist()
counties.pop()
counties

['Bell',
 'Bexar',
 'Bowie',
 'Brazoria',
 'Brazos',
 'Cameron',
 'Collin',
 'Crane',
 'Dallas',
 'Denton',
 'DeWitt',
 'El Paso',
 'Ellis',
 'Fannin',
 'Fort Bend',
 'Galveston',
 'Gregg',
 'Harris',
 'Hays',
 'Hockley',
 'Johnson',
 'Lavaca',
 'Lubbock',
 'Matagorda',
 'McLennan',
 'Medina',
 'Montgomery',
 'Rusk',
 'Smith',
 'Tarrant',
 'Travis',
 'Webb',
 'Wichita',
 'Williamson']

In [ ]:
# Scrape the Lat Long data

lats = []
longs = []
for county in counties:
    browser.visit(url2)
    
    browser.fill('q', f'{county} county Texas Lattitude Longitude')
    
    time.sleep(2)

    enter = browser.find_by_name('q')
    
    enter.type(Keys.RETURN)
    
    time.sleep(2)
    
    geo_info = browser.find_by_css('.Z0LcW').text
    
    lat = geo_info.split('°')[0]
    long = geo_info.split(',')[1].split('°')[0].strip()
    
    lats.append(float(lat))
    longs.append(-float(long))
    
    
    
    

In [ ]:
Counties_Pending = df.tail(1)

In [ ]:
Counties_Pending

In [ ]:
df.drop(df.tail(1).index,inplace=True)

In [ ]:
# Adding lat and longs to the dataframe
df['lat'] = lats
df['long'] = longs

In [ ]:
df

In [ ]:
browser.quit()

In [ ]:

# browser.visit(url2)
# browser.fill('q', f'{counties[0]} county Texas Lattitude Longitude')

# enter = browser.find_by_name('q')

In [ ]:
# enter.type(Keys.RETURN)

In [ ]:
# Bell_country = browser.find_by_css('.Z0LcW').text
# Bell_country

In [ ]:
# lat = Bell_country.split('°')[0]
# long = Bell_country.split(',')[1].split('°')[0].strip()

In [ ]:
def df_to_geojson(df, properties, lat='lat', lon='long'):
    prefix = 'var' + ' data' + ' = '
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point','coordinates':[]}}
        feature['geometry']['coordinates'] = [row[lon],row[lat]]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

cols = ['Texas_County', 'COVID_19_Cases']
javascript_file = df_to_geojson(df, cols)
# {'type': 'FeatureCollection', 'features': [{'geometry': {'type': 'Point', 'coordinates': [-70.5867, -29.9953, 760]}, 'type': 'Feature', 'properties': {'name': 'A', 'description': 'Place a'}}, {'geometry': {'type': 'Point', 'coordinates': [-70.4933, -30.1217, 1250]}, 'type': 'Feature', 'properties': {'name': 'B', 'description': 'Place b'}}, {'geometry': {'type': 'Point', 'coordinates': [-70.5008, -30.0953, 1185]}, 'type': 'Feature', 'properties': {'name': 'C', 'description': 'Place c'}}]}

In [ ]:

with open('static/js/map.json','w') as f:
    json.dump(javascript_file, f, indent=4, sort_keys=False)